# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
hygiene_query = Queries(establishments,[{'scores.Hygiene':{"$eq":20}}])

hygiene_query_results = establishments.find(hygiene_query)
# Use count_documents to display the number of documents in the result
hygiene = establishments.count_documents({'scores.Hygiene': {'$eq': 20}})
print('Total establishments with a hygiene score of 20: ', hygiene)
print()

# Display the first document in the results using pprint
hygiene_query.get_result(0)

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene_query.ConvertToDF()
# Display the number of rows in the DataFrame
print("DataFrame Rows: ", len(hygiene_query_df))
# Display the first 10 rows of the DataFrame
hygiene_query_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
rating_value_query = Queries(establishments,[{"LocalAuthorityName" : {"$regex" : "London"} , "RatingValue" : {"$gte":"4"}}])

rating_value_results = establishments.find(rating_value_query)

# Use count_documents to display the number of documents in the result
rating = establishments.count_documents(rating_value_query)

print('Total establishments in London with a rating of 4 or greater: ', rating)
print()

# Display the first document in the results using pprint
pprint(rating_value_results[0])

In [ ]:
# Convert the result to a Pandas DataFrame #hygiene_query.ConvertToDF()
rating_df = pd.DataFrame(rating_value_results) 
# Display the number of rows in the DataFrame
print("DataFrame Rows: ", len(hygiene_query_df))
# Display the first 10 rows of the DataFrame
rating_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = establishments.find_one({"BusinessName" : "Penang Flavours"})["geocode"]["latitude"]
longitude = establishments.find_one({"BusinessName" : "Penang Flavours"})["geocode"]["longitude"]



look_query=Queries(establishments,[{'RatingValue' : {'$regex' : '5'},\
                                       'geocode.latitude' : {'$lte': add[0], '$gte': sub[0]},\
                                       'geocode.longitude': {'$lte': add[1], '$gte': sub[1]}}])
look_query.sort_limit('scores.Hygiene', 1, 5)
# Print the results
print()

In [ ]:
# Convert result to Pandas DataFrame
look_query.ConvertToDF()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipe_query = Queries(establishments, [{'$match':{'scores.Hygiene':{"$eq":0}}},\
                                     
                                      {'$group':{'_id':'$LocalAuthorityName','count':{'$sum': 1}}},\
                                     
                                      {'$sort' :{'count':-1}}])
# Print the number of documents in the results
print("Total establishments within each LocalAuthority with a hygiene score of '0': ",(pipe_query.set_result))
# Print the first 10 results

print(pipe_query.get_result(10))

In [ ]:
# Convert the result to a Pandas DataFrame
pipe_query.ConvertToDF()
# Display the number of rows in the DataFrame
print(pipe_query.set_result)
# Display the first 10 rows of the DataFrame
pipe_query.set_result.head(10)